In [137]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [138]:
df = pd.read_parquet('../Proyecto/Experimentos/Clase6_PredJulio2018/05_sell_in_pred.parquet')

In [139]:
df

,product_id,periodo,primer_periodo,ultimo_periodo,tn_norm,periodo_dt,mes,quarter,month_in_quarter,year,...,prop_product_yearquarter_brand,prop_product_year_cat1,prop_product_year_cat2,prop_product_year_brand,tn_futuro,tn_prediccion,average_tn,std_dev_tn,tn_futuro_real,tn_prediccion_real
0,20001,201807,201701,201912,0.241713,201807,7,3,1,2018,...,23.302392,0.661392,1.282107,5.945263,-0.106443,-0.080946,1398.344322,298.145460,1438.67455,1446.276310
1,20002,201807,201701,201912,-0.105208,201807,7,3,1,2018,...,4.181841,0.439637,0.852235,0.993159,-0.076247,0.111377,1009.368178,303.834835,954.23575,1011.242701
2,20003,201807,201701,201912,-0.797082,201807,7,3,1,2018,...,5.651830,0.870366,1.185009,1.549996,0.876995,0.983980,889.004243,292.036581,912.34156,943.585108
3,20004,201807,201701,201912,-0.081053,201807,7,3,1,2018,...,5.627681,0.866647,1.179946,1.543373,1.316285,1.119871,671.615383,224.450085,948.86342,904.778326
4,20005,201807,201701,201912,-0.649918,201807,7,3,1,2018,...,4.326467,0.666264,0.907123,1.186520,1.188577,1.060158,644.200514,218.273222,761.77520,733.744768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,21207,201807,201711,201912,0.078208,201807,7,3,1,2018,...,0.311524,0.000323,0.068418,0.068418,-0.721815,-0.423219,0.202266,0.121014,0.12438,0.160514
599,21212,201807,201711,201912,-0.134082,201807,7,3,1,2018,...,0.001948,0.000076,0.000591,0.000466,-0.395354,-0.360355,0.183247,0.110205,0.12490,0.128757
600,21218,201807,201701,201912,0.366040,201807,7,3,1,2018,...,2.435302,0.751286,0.751286,0.751286,-0.672458,-0.527121,0.119496,0.130462,0.07952,0.098481
601,21222,201807,201701,201912,-0.092941,201807,7,3,1,2018,...,1.431186,0.441518,0.441518,0.441518,-0.320933,-0.148312,0.108623,0.111176,0.06261,0.081801


In [140]:
df_pred = df[['product_id', 'tn_futuro_real', 'tn_prediccion_real']]
df_pred['error'] = -(df_pred['tn_futuro_real'] - df_pred['tn_prediccion_real'])/df_pred['tn_futuro_real']*100
df_pred['errorabs'] = abs(df_pred['tn_futuro_real'] - df_pred['tn_prediccion_real'])
df_pred['weight'] = df_pred['tn_futuro_real']/df_pred['tn_futuro_real'].sum()
df_pred

,product_id,tn_futuro_real,tn_prediccion_real,error,errorabs,weight
0,20001,1438.67455,1446.276310,0.528386,7.601760,0.050160
1,20002,954.23575,1011.242701,5.974095,57.006951,0.033270
2,20003,912.34156,943.585108,3.424545,31.243548,0.031809
3,20004,948.86342,904.778326,-4.646095,44.085094,0.033083
4,20005,761.77520,733.744768,-3.679620,28.030432,0.026560
...,...,...,...,...,...,...
598,21207,0.12438,0.160514,29.051607,0.036134,0.000004
599,21212,0.12490,0.128757,3.088052,0.003857,0.000004
600,21218,0.07952,0.098481,23.844301,0.018961,0.000003
601,21222,0.06261,0.081801,30.652025,0.019191,0.000002


In [141]:
abs(df_pred['tn_futuro_real'] - df_pred['tn_prediccion_real']).sum()/df_pred['tn_futuro_real'].sum()

0.08029646305456453

In [142]:
prod_data = pd.read_csv('../tb_productos.txt', sep='\t')

In [143]:
df_cat = df_pred.merge(prod_data, on='product_id', how='left')

In [144]:
df_cat_group =df_cat[['cat1', 'tn_futuro_real', 'tn_prediccion_real', 'errorabs']].groupby('cat1').sum()

In [145]:
df_cat_group['Err'] = df_cat_group['errorabs']/df_cat_group['tn_futuro_real']
df_cat_group

,tn_futuro_real,tn_prediccion_real,errorabs,Err
cat1,,,,
FOODS,6811.37762,6754.534910,454.275546,0.066694
HC,18630.22378,18886.864025,1641.442309,0.088106
PC,4648.01828,4727.135297,393.551130,0.084671
REF,2.13798,1.983302,0.230982,0.108038


In [146]:
df = pd.read_csv('../sell-in.txt',sep='\t')
prod_a_predecir = pd.read_csv('../productos_a_predecir.txt', sep='\t')
prod_data = pd.read_csv('../tb_productos.txt', sep='\t')

In [147]:
# for col in prod_data.select_dtypes(include=['object']).columns:
#     prod_data[col] = prod_data[col].astype('category')

# prod_data = prod_data.drop_duplicates(subset='product_id')

In [160]:
df_grouped = df.groupby(['periodo', 'product_id']).sum().reset_index()

In [161]:
prod_stats = df_grouped.groupby('product_id').apply(lambda group: pd.Series({
    'primer_periodo': group['periodo'].min(),
    'ultimo_periodo': group['periodo'].max(),
    'values': group['tn'].count(),
    'total_tn': group['tn'].sum(),
    'min_tn': group['tn'].min(),
    'average_tn': group['tn'].mean(),
    'median_tn': group['tn'].median(),
    'std_dev_tn': group['tn'].std(),
    'iqr_tn': (group['tn'].quantile(0.75) - group['tn'].quantile(0.25)),
    'max_tn': group['tn'].max()
}))
prod_stats['primer_periodo'] = prod_stats['primer_periodo'].astype(int)
prod_stats['primer_periodo'] = pd.to_datetime(prod_stats['primer_periodo'].astype(str), format='%Y%m')

prod_stats['ultimo_periodo'] = prod_stats['ultimo_periodo'].astype(int)
prod_stats['ultimo_periodo'] = pd.to_datetime(prod_stats['ultimo_periodo'].astype(str), format='%Y%m')

prod_stats.reset_index(inplace=True)

In [162]:
df_grouped = df_grouped[['product_id', 'periodo', 'tn']]
df_grouped.sort_values(by=['product_id', 'periodo'], inplace=True)
df_grouped.reset_index(drop=True, inplace=True)

In [163]:
print(df_grouped.shape)
df_grouped.head()

(31243, 3)


,product_id,periodo,tn
0,20001,201701,934.77222
1,20001,201702,798.01620
2,20001,201703,1303.35771
3,20001,201704,1069.96130
4,20001,201705,1502.20132


In [151]:
prod_stats = df_grouped.groupby('product_id').apply(lambda group: pd.Series({
    'primer_periodo': group['periodo'].min(),
}))

prod_stats.reset_index(inplace=True)

In [152]:
df_doce = df_grouped[(df_grouped['periodo'] >= 201706) & (df_grouped['periodo'] <= 201805)].groupby('product_id').sum()
df_doce.drop(['periodo'], axis=1, inplace=True)
df_doce.reset_index(inplace=True)
df_doce.columns = ['product_id', 'tn_doce']
df_doce['tn_doce'] = df_doce['tn_doce']/12


In [153]:
df_julio = df_grouped[df_grouped['periodo'] == 201807].groupby('product_id').sum()
df_doce = df_doce.merge(df_julio, on='product_id', how='left')
df_doce

,product_id,tn_doce,periodo,tn
0,20001,1318.296338,201807.0,1470.41009
1,20002,891.111009,201807.0,977.40239
2,20003,1019.168982,201807.0,656.22700
3,20004,738.165472,201807.0,653.42310
4,20005,663.494222,201807.0,502.34077
...,...,...,...,...
958,21293,0.000764,NaN,NaN
959,21294,0.000637,NaN,NaN
960,21296,0.000543,NaN,NaN
961,21298,0.000478,NaN,NaN


In [154]:
abs(df_doce['tn_doce'] - df_doce['tn']).sum()/df_doce['tn'].sum()

0.3480591934373917

In [155]:
df_doce = df_doce.merge(prod_data, on='product_id', how='left', suffixes=['',''])
df_doce

,product_id,tn_doce,periodo,tn,cat1,cat2,cat3,brand,sku_size
0,20001,1318.296338,201807.0,1470.41009,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
1,20002,891.111009,201807.0,977.40239,HC,ROPA LAVADO,Liquido,LIMPIEX,3000.0
2,20003,1019.168982,201807.0,656.22700,FOODS,ADEREZOS,Mayonesa,NATURA,475.0
3,20004,738.165472,201807.0,653.42310,FOODS,ADEREZOS,Mayonesa,NATURA,240.0
4,20005,663.494222,201807.0,502.34077,FOODS,ADEREZOS,Mayonesa,NATURA,120.0
...,...,...,...,...,...,...,...,...,...
968,21293,0.000764,NaN,NaN,NaN,NaN,NaN,NaN,NaN
969,21294,0.000637,NaN,NaN,PC,CABELLO,Tratamiento Fem,NIVEA,350.0
970,21296,0.000543,NaN,NaN,NaN,NaN,NaN,NaN,NaN
971,21298,0.000478,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [156]:
df_doce['errorabs'] = abs(df_doce['tn'] - df_doce['tn_doce'])

In [157]:
df_doce_group =df_doce[['cat1', 'tn', 'tn_doce', 'errorabs']].groupby('cat1').sum()

In [158]:
df_doce_group

,tn,tn_doce,errorabs
cat1,,,
FOODS,7073.33599,8415.941898,2366.431012
HC,21789.43088,27194.085241,7658.248830
PC,6219.04821,7938.378356,2570.849312
REF,3.39907,2.521540,1.066565


In [159]:
df_doce_group['Err'] = df_doce_group['errorabs']/df_doce_group['tn']
df_doce_group

,tn,tn_doce,errorabs,Err
cat1,,,,
FOODS,7073.33599,8415.941898,2366.431012,0.334557
HC,21789.43088,27194.085241,7658.248830,0.351466
PC,6219.04821,7938.378356,2570.849312,0.413383
REF,3.39907,2.521540,1.066565,0.313781
